In [ ]:
# load data
from data_utils import load_ucr_archive, load_ucr_dataset

data = load_ucr_archive()
#data.drop(data[data["dataset"] == "ACSF1"].index, inplace=True)

#data = load_ucr_dataset(48)
#data.head()

In [ ]:
from spline_utils import calculate_max_dist


def calculate_error_measures_splines(knots, coeffs, degree, time_series):
    x_values = [x[0] for x in time_series]
    y_values = [x[1] for x in time_series]
    fitted_spline = [evaluate_spline(knots, result, degree, x) for x in x_values]
    max_dist = calculate_max_dist(knots, coeffs, degree, time_series)
    mse = mean_squared_error(fitted_spline, y_values)
    mae = mean_absolute_error(fitted_spline, y_values)
    return [max_dist, mse, mae]


In [ ]:
from fit import fit_PAA, fit_DFT


def calculate_compression_methods(time_series, knots, degree):
    least_squares = scipy.interpolate.make_lsq_spline([x[0] for x in time_series], [x[1] for x in time_series], knots,
                                                      k=degree).c
    max_dist, l_8 = fit_max_spline(time_series, knots, degree)
    l_1 = fit_max_l1_spline(time_series, knots, degree, max_dist)

    # competitors
    num_coeffs = len(knots) - degree - 1
    paa = fit_PAA(time_series, num_coeffs)
    dft = fit_DFT(time_series, num_coeffs)


In [ ]:
def compare_compression_methods(coeffs, methods):
    
    assert(len(coeffs) == len(fitting_methods))
    
    # TODO aufräumen
    result = []
    for i, coeff in enumerate(coeffs):
        next_row = [row['dataset'], row['no.'], len(time_series), num_outliers, fitting_methods[i]]
        next_row.extend(calculate_error_measures_splines(knots, coeff, degree, time_series))

    

In [ ]:
%%time
from data_utils import remove_outliers
import scipy
from spline_utils import generate_knot_vector_from_coeff_count, generate_coeff_counts
from fit import fit_max_spline, fit_max_l1_spline
from sklearn.metrics import mean_squared_error, mean_absolute_error
from spline_utils import evaluate_spline
import pandas as pd

results = []
num_outliers = 0
fitting_methods = [r'$L_2$', r'$L_{\infty}$', r'$L_{\infty}$ and $L_1$', 'PAA', 'SAX', 'DFT']
fst_diff = snd_diff = []
for index, row in data.iterrows():
    time_series = row['data']
    print(row['dataset'], "no.", row['no.'])

    # generate the number of knots according to the specified compression rate
    num_knots = generate_coeff_counts(len(time_series), degree, [compression_ratio])[0]
    knots = generate_knot_vector_from_coeff_count(num_knots, degree)

    ts_lof = remove_outliers(time_series)
    # calculate results for each fitting method
    num_outliers = 0

    results = []
    # LSQ
    results.append(
        scipy.interpolate.make_lsq_spline([x[0] for x in time_series], [x[1] for x in time_series], knots, k=degree).c)
    max_dist, result_max = fit_max_spline(time_series, knots, degree)
    
    # L8
    results.append(result_max)
    
    # L8-L1
    results.append(fit_max_l1_spline(time_series, knots, degree, max_dist))
    
    # PAA
    
    
    # SAX
    
    # DFT
    
    assert(len(results) == len(fitting_methods))
    
    for i, result in enumerate(results):
        next_row = [row['dataset'], row['no.'], len(time_series), num_outliers, fitting_methods[i]]
        next_row.extend(calculate_error_measures_splines(knots, result, degree, time_series))
        results.append(next_row)
        

In [ ]:
df = pd.DataFrame(results,
                  columns=['dataset_name', 'ts_number','num_data_pts', 'num_outliers', 'fitting_method', 'max_dist', 'MSE', 'MAE'])#, 'fst_diff', 'snd_diff'])
df